Our starting point is `batman.zip` file which contains one folder per mesh node. Each of the folders contain files under four headings, each file containing data from a single day.

First, we unzip all of into a folder `mesh_logs`.

In [1]:
import os

os.mkdir('mesh_logs')
os.system('unzip batman.zip')
os.system('mv batman/* mesh_logs')
os.system('rm -r batman')

0

Next we convert the folder names to standard IP format. This is important for two reasons 1) IP format is an actual standard and there is no case for not using that standard 2) we can this way directly use the folder name as meta-data in the datatables we create in later steps

In [2]:
# get the list of the nodes
nodes = os.popen('ls mesh_logs').readlines()

# convert the names to standard ip format
for node in nodes:
    
    current = node.strip()
    new = node.strip().replace('_', '.')

    os.system('mv mesh_logs/' + current + ' mesh_logs/' + new)

Next, we decompress the .gz files.

In [3]:
import gzip

# first we get the list of the node IPs
for node in os.popen('ls mesh_logs').readlines():
    
    # remove newlines
    node = node.strip()
    
    os.system('gzip -d mesh_logs/' + node + '/*.gz')

Then we move on to combine all the individual files into single files following the breakdown to four types of files: 

- ping_stats
- mesh
- bat
- wifi

We will change the naming of these to:

- ping
- mesh
- routing
- wifi

Lastly before moving onto creating the datatables, we unify all the files in each node under the four headings. This leaves us four files per node.

In [4]:
for node in os.popen('ls mesh_logs').readlines():
    
    # remove newlines
    node = node.strip()
    
    os.system('cat mesh_logs/' + node + '/bat*.lo* > ' + node + '_routing.log')
    os.system('cat mesh_logs/' + node + '/mesh*.lo* > ' + node + '_mesh.log')
    os.system('cat mesh_logs/' + node + '/ping_stats*.lo* > ' + node + '_ping.log')
    os.system('cat mesh_logs/' + node + '/wifi*.lo* > ' + node + '_wifi.log')

Before moving on, let's put all the files into a single directory `cleaned_logs` and delete the original files.

In [5]:
os.mkdir('cleaned_logs')
os.system('mv *.log cleaned_logs')
os.system('rm -rf mesh_logs/')

0

In [18]:
def wifi_clients(node_ip):

    f = open('cleaned_logs/' + node_ip + '_wifi.log', 'r')
    out = f.readlines()

    final = []

    for i in out:

        i = i.strip()

        if i.startswith('###'):
            date = ' '.join(i.split()[2:])
        elif 'Number' in i:
            data = i.split(':')[-1]
            final.append([node_ip, date, data])
            
    return final

In [51]:
nodes = []

for i in os.popen('ls cleaned_logs').readlines():
    i = i.strip()
    nodes.append(i.split('_')[0])
    nodes = list(set(out))

In [53]:
out = []

for node in nodes:
    
    data = wifi_clients(node)
    
    out += data

Now we can create a pandas dataframe from the data, and convert numeric values to ints and the timestamp to an actual datetime index.

In [60]:
import pandas as pd

wifi_df = pd.DataFrame(out)
wifi_df.columns = ['node_ip', 'timestamp', 'no_of_connections']
wifi_df.no_of_connections = wifi_df.no_of_connections.astype(int)
wifi_df.timestamp = pd.DatetimeIndex(wifi_df.timestamp)

wifi_df.to_csv('logmesh_zanskar_wifi.csv', index=None)